# Entendiendo los contextos

Esta es la mejor introduccion que he visto a los context managers. Este es un concepto un poco complicado para verlo por pirmera vez.

Fuente: <a href="https://www.youtube.com/watch?v=Lv1treHIckI">Video</a>

Paso a resumir el video:

1) Es importante entender el problema que queremos resolver:


In [17]:
from pathlib import Path
# Abrir y cerrar un archivo

file = open(Path.cwd()/"disci"/"repaso_2024/contextos_archivos/file.txt","w") # abro el archivo
file.write("hello world") # escribo en el 
file.close() # Lo cierro

/home/jorgefederico/disci


El tema es el siguiente, si hay algun error en la parte de escritura, vos te quedas con el archivo abierto, por lo cual me gustaria ver una manera de cerrarlo sin importar lo que pase. Por lo cual, agrego un bloque try, finally.

In [18]:
file = open(Path.cwd()/"disci"/"repaso_2024/contextos_archivos/file.txt","w") # abro el archivo
try:
    file.write("hello world") # escribo en el 
finally: #Ya sabemos que esto se va a ejecutar pase lo que pase
    file.close() # Lo cierro

Como siguiente paso quisieramos hacer esto de forma mas corta y esto implica usar un manejador de contexto, que es la palabra with. El codigo siguiente es exactamente lo mismo que teniamos arriba.

In [19]:
with open(Path.cwd()/"disci"/"repaso_2024/contextos_archivos/file.txt","w") as file:
    file.write("hello wold")

Ahora vamos a escribir nuestro propip context manager. Existen dos metodos que debemos agregar a nuestra clase para convertirla en un manejador de contexto:

- `__enter__()`: Retorna siempre un valor que es el que se le pasa a la palabra designada luego de `as`.
- `__exit__()` : Este bloque nos va a permitir manejar excepciones si aparecen ademas de ejecutarse como una especie de finllay.


In [20]:
class File:
    def __init__(self, filename, method):
        self.file = open(filename, method)
    # Primer metodo
    def __enter__(self):
        print('Enter') # Agregamos estos prints para ver en que orden se ejecutan las cosas
        return self.file 
    # Segundo metodo
    def __exit__(self,type,value,traceback):
        print('Exit') # Print para ver que sucede
        self.file.close() # Aca le decimos que se cierre pase lo que pase

In [23]:
with File(Path.cwd()/"disci"/"repaso_2024/contextos_archivos/file.txt","w") as file:
    # Aca hago cosas con el objeto file que me retorna la clase File
    print("Middle")
    file.write("Pikachu")
    raise Exception # Veamos que el bloque exit se ejecuta aunque aparezca un error

Enter
Middle
Exit


Exception: 

Ahora vamos a ir un paso mas alla y vamos a ver de que manera podemos manejar nuestra excepcion:

In [34]:
class File:
    def __init__(self, filename, method):
        self.file = open(filename, method)
    # Primer metodo
    def __enter__(self):
        print('Enter') # Agregamos estos prints para ver en que orden se ejecutan las cosas
        return self.file 
    # Segundo metodo
    def __exit__(self,type,value,traceback):
        print(f"type:<{type}> value:<{value}> traceback:<{traceback}>")
        print('Exit') # Print para ver que sucede
        self.file.close() # Aca le decimos que se cierre pase lo que pase
        # Aca ventdria el manejo de excepcion
        if Exception:
            print("Hubo una excepcion")
            return True # Retornar true, le indica a python que usted manejo la excepcion y esta no se mostrara en ejecucion


In [35]:
with File(Path.cwd()/"disci"/"repaso_2024/contextos_archivos/file.txt","w") as file:
    # Aca hago cosas con el objeto file que me retorna la clase File
    print("Middle")
    file.write("Pikachu")
    raise Exception # Veamos que el bloque exit se ejecuta aunque aparezca un error

Enter
Middle
type:<<class 'Exception'>> value:<> traceback:<<traceback object at 0x7f6a481f2340>>
Exit
Hubo una excepcion


Unas ultimas observaciones
- Si exit retorna True, significa que se manejo la excepcion
- Si exit retorna None, False se propaga la excepcion

## Creando un manejador de contextos con context manager

- utilizamos: contextlib.contextmanager como decorador sobre nuestra funcion
- La funcion debe ser transformada a genearador por lo cual debe tener yield en lugar de return

In [37]:
@contextlib.contextmanager
def file(filename,method):
    #__enter__()
    print('Enter')
    file = open(filename,method)
    yield file
    #__exit__() , aca es donde tambien manejo los errores
    file.close()
    print('Exit')


In [39]:
with file(Path.cwd()/"disci"/"repaso_2024/contextos_archivos/file.txt","w") as f:
    print("middle")
    f.write("contenido")

Enter
middle
Exit


Notas:
- Todo lo que este antes del yield sera ejecutado al principio del bloque with en cuanto el interprete llame a `__enter__`
- El codigo despues de  yield sera ejecutado cuando el interprete llame a `__exit__`

Veamos como finalmente realizar un manejo de errores:

In [44]:
@contextlib.contextmanager
def file(filename,method):
    #__enter__()
    print('Enter')
    file = open(filename,method)
    try:
        yield file
    except Exception:
        print("Hubo una excepcion")
    #__exit__() , aca es donde tambien manejo los errores
    # finally:
    file.close()
    print('Exit')


In [45]:
with file(Path.cwd()/"disci"/"repaso_2024/contextos_archivos/file.txt","w") as f:
    print("middle")
    f.write("contenido")
    raise Exception

Enter
middle
Hubo una excepcion
Exit


# Manejadores de contextos como decoradores

- Deacuerdo con el libro de Fluent python cualquier cosa decorada con @contextmanager se puede utilizar como decorador.

Consideremos el siguiente manejador de contexto del libro (Basicamente es un reverseador de palabras, toma una palabra y la escribe alrevez):


In [52]:
import sys
@contextlib.contextmanager
def looking_glass():
    original_write = sys.stdout.write
    def reverse_write(text):
        original_write(text[::-1])
    sys.stdout.write = reverse_write
    msg = ''
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)

In [53]:
@looking_glass()
def verse():
    print("a beautiful verse")

verse()

esrev lufituaeb a


# chdir de contextlib

Contextlib tiene otros usos, copio lo que dice Juan:

- closing(thing): Crea un context manager que cierra thing al salir del contexto.
- suppress(*exceptions): Context manager para suprimir excepciones específicas.
- redirect_stdout(new_target) y redirect_stderr(new_target): Redirigen temporalmente la salida estándar o de error.
- chdir(path) Cambia temporalmente el directorio de trabajo actual al especificado por path al entrar en el contexto, y automáticamente restaura el directorio de trabajo original al salir del contexto. <--- Esta ya la utilize antes y permite cambiar de directorio utilizando contextos

## Mas importante, Cuando utilizar esto?

- Gestión de recursos: Para manejar la adquisición y liberación automática de recursos como archivos, conexiones de red o bases de datos.
- **Configuración temporal**: Para establecer un estado temporal y restaurarlo automáticamente, como cambiar directorios de trabajo o modificar variables de entorno.
- Manejo de transacciones: Para operaciones que deben completarse en su totalidad o revertirse, como transacciones de base de datos u operaciones atómicas de archivo.
- Bloqueos y sincronización: Para gestionar la exclusión mutua en recursos compartidos, como bloqueos de threads o semáforos.
- **Medición y logging**: Para rastrear tiempos de ejecución o registrar eventos de entrada/salida de manera automática. *Podriamos probar crear un contexto para medir el tiempo de ejecuion de mi programa.*